In [1]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
link = 'https://books.toscrape.com/catalogue/page-1.html'
res = requests.get(link)

In [3]:
soup = BeautifulSoup(res.text, 'html.parser')

In [9]:
details = []
for i in range(1, 51):
    link = 'https://books.toscrape.com/catalogue/page-'+str(i)+'.html'
    res = requests.get(link)
    soup = BeautifulSoup(res.text, 'html.parser')
    for sp in soup.find_all('li', class_ = 'col-xs-6 col-sm-4 col-md-3 col-lg-3'):
        link = "https://books.toscrape.com/catalogue/"+sp.find_all('a')[-1].get('href')
        title = sp.find_all('a')[-1].get('title')
        price = sp.find('p', class_ = 'price_color').text[1:]
        availability = sp.find('p', class_ = 'instock availability').text.strip()
        img_link = "https://books.toscrape.com"+sp.find('img').get('src')[2:]
        rating = sp.find('p').get('class')[-1]
        details.append([link, title, img_link, price, rating, availability])

In [11]:
df = pd.DataFrame(details, columns=['Link', 'Title', 'Img_Link', 'Price', 'Ratings', 'Availability'])
df.to_csv('BooksDetails.csv', index = False)

In [12]:
df.head()

,Link,Title,Img_Link,Price,Ratings,Availability
0,https://books.toscrape.com/catalogue/a-light-i...,A Light in the Attic,https://books.toscrape.com/media/cache/2c/da/2...,£51.77,Three,In stock
1,https://books.toscrape.com/catalogue/tipping-t...,Tipping the Velvet,https://books.toscrape.com/media/cache/26/0c/2...,£53.74,One,In stock
2,https://books.toscrape.com/catalogue/soumissio...,Soumission,https://books.toscrape.com/media/cache/3e/ef/3...,£50.10,One,In stock
3,https://books.toscrape.com/catalogue/sharp-obj...,Sharp Objects,https://books.toscrape.com/media/cache/32/51/3...,£47.82,Four,In stock
4,https://books.toscrape.com/catalogue/sapiens-a...,Sapiens: A Brief History of Humankind,https://books.toscrape.com/media/cache/be/a5/b...,£54.23,Five,In stock


##### Extracting Information from individual book site

In [13]:
df = pd.read_csv('BooksDetails.csv')

In [ ]:
df['Link'].head()

0    https://books.toscrape.com/catalogue/a-light-i...
1    https://books.toscrape.com/catalogue/tipping-t...
2    https://books.toscrape.com/catalogue/soumissio...
3    https://books.toscrape.com/catalogue/sharp-obj...
4    https://books.toscrape.com/catalogue/sapiens-a...
Name: Link, dtype: object

In [56]:
bdetails = []
for link in tqdm(df['Link']):
    res = requests.get(link)
    soup = BeautifulSoup(res.text, 'html.parser')
    category = soup.find('ul', class_ = 'breadcrumb').find_all('a')[2].text
    bdetail = soup.find('table', class_ = 'table table-striped').find_all('td')
    UPC = bdetail[0].text
    price_exctax = bdetail[2].text[1:]
    price_inctax = bdetail[3].text[1:]
    tax = bdetail[4].text[1:]
    in_stock = bdetail[5].text
    start_pos = in_stock.find('(') + 1 
    no_available_books = in_stock[start_pos:-11].strip()
    no_reviewes = bdetail[6].text
    bdetails.append([category, UPC, price_exctax, price_inctax, tax, in_stock, no_available_books, no_reviewes])

100%|██████████| 1000/1000 [30:42<00:00,  1.84s/it]


In [58]:
df = pd.DataFrame(bdetails, columns=['category', 'UPC', 'price_e_tax', 'price_i_tax', 'tax', 'in_stock', 'no_of_availability', 'no_reviews'])

In [61]:
df.drop('in_stock', axis = 1, inplace = True)

In [62]:
df.head()

,category,UPC,price_e_tax,price_i_tax,tax,no_of_availability,no_reviews
0,Poetry,a897fe39b1053632,£51.77,£51.77,£0.00,22,0
1,Historical Fiction,90fa61229261140a,£53.74,£53.74,£0.00,20,0
2,Fiction,6957f44c3847a760,£50.10,£50.10,£0.00,20,0
3,Mystery,e00eb4fd7b871a48,£47.82,£47.82,£0.00,20,0
4,History,4165285e1663650f,£54.23,£54.23,£0.00,20,0


In [64]:
df.to_csv('IndividualBookDetails.csv', index = False)

##### Concatenate two dataframe

In [65]:
df1 = pd.read_csv('BooksDetails.csv')
df2 = pd.read_csv('IndividualBookDetails.csv')

In [70]:
resultant_df = pd.concat([df1, df2], axis = 1)
# resultant_df.head()
resultant_df.shape


(1000, 13)

In [71]:
resultant_df.to_csv('AllDetails.csv', index=False)